In [ ]:
from traitlets.config.application import default
from pandas.core.indexes import category
import os
import csv
import pandas as pd
from collections import defaultdict
class ExtractingEvaluationData(object):

    categories = ['purpose','description','scope','requirements','assumptions and dependencies','architecture','users']
    path = os.getcwd()+'/../Data'

    def __init__(self):
        print('ExtractingEvaluationData Results...')
        contentExp = self.getTestData()
        contentPred, srs = self.getPredictedData(self.categories)
        df = self.processData(contentPred, contentExp, srs, self.categories)
        self.writeData(df)

        
    def getTestData(self):
        test_data = defaultdict(dict)
        read_path = self.path +'/evaluation_data/expected_data.csv'
        df = pd.read_csv(read_path)
        df = df.fillna(' ')
        
        for i, row in df.iterrows():
            test_data[row[0]][row[1]] = row[2]
        return test_data

    def getPredictedData(self,categories):
        contentPred = defaultdict(dict)
        srs = []
        dest = self.path+'/summarized_data/summaries'
        
        for category in categories:
            read_path = dest+'/'+category+'/'
            documents = os.listdir(read_path)
            for doc in documents:
                with open(read_path+doc, 'r') as input:
                    doc_num = doc[:-4]

                    contentPred[doc_num][category] = str(input.read())

        for doc in documents:
            doc = doc.split('/')[-1]
            srs.append(doc[:-4])

        return contentPred,srs

    def processData(self,contentPred, contentExp, srs, categories):
        print(contentPred,'\nexp:',contentExp)
        df = pd.DataFrame(columns=['doc','predicted','expected'])
        docs = []
        pred = []
        expect = []
        for s in srs:
            for category in categories:
                docs.append(s)
                if len(contentPred[s])!=0:
                    v1 = contentPred[s][category]
                else:
                    v1 =''
                if len(contentExp[s])!=0:
                    v2 = contentExp[s][category]
                else:
                    v2 =''
                pred.append(v1) 
                expect.append(v2)
        df['doc'] = docs
        df['predicted'] = pred
        df['expected'] = expect
        display(df)
        return df

    def writeData(self,df):
        write_path = self.path +'/evaluation_data/evaluation_data.csv'
        # Create CSV - add Dataframe
        df.to_csv(write_path, index=False, sep=',')

# ExtractingEvaluationData()